# FOM - Anwendungsfelder der Business Analytics am Beispiel von Fraud Detection im von Kreditkartentransaktionen
In diesem Jupyter Note soll ein künstliches neuronales Netz (KNN) erstellt werden, mit welchem Kreditkartentransaktionen auf betrügerische Aktivitäten hin überprüft werden. Für die Umsetzung dieses Vorhabens sollen die in dem Notebook <code>ieee-cis-k-means</code> erstellten Daten und Tensorflow verwendet werden.</br></br>
<b>Folgende Schritte sind zu erledigen</b>
<ol>
    <li>Import der nötigen Python Packages</li>
    <li>Import der vorbereiteten Daten aus AWS S3</li>
    <li>Normalisierung der importierten Daten</li>
</ol>
</br>
Im folgenden werden die zu verwenden Python Packages importiert.

In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals
import boto3
import io
import matplotlib.pyplot as plt
import numpy as py
import pandas as pd
import tensorflow as tf

In [3]:
print('{}: {}'.format('Tensorflow version', tf.__version__))

Tensorflow version: 1.14.0


Nachdem die nötigen Python Packages importiert sind, können die zuletzt vorbereiteten Daten aus AWS S3 importiert werden.
<br>
## Import der vorbereiteten Daten aus AWS S3
Für den Import von Daten aus AWS S3 muss zunächst der AWS S3 client initialisiert und der entsprechende Bucket Name angegeben werden.

In [5]:
s3_client = boto3.client('s3')
bucket_name = 'anwendungsfelder-big-data-fraud-detection-ieee-cis-data-set'

Anschließend wird eine Iteration über alle in dem AWS S3 Bucket enthaltende Objekte vorgenommen.

In [6]:
obj_list = s3_client.list_objects(Bucket=bucket_name)
for obj in obj_list:
    print(obj)

Name
ResponseMetadata
MaxKeys
Prefix
Marker
EncodingType
IsTruncated
Contents


Da der Inhalt des Buckets benötigt wird, wird eine Iteration über den Inhalt des Buckets durchgeführt. Da aber das aus der Abfrage zurückkommende Objekt nicht bekannt ist, wird zunächst nur ein Element aus der Liste ausgegeben.

In [10]:
content_list = obj_list['Contents']
print(content_list[0])

{u'LastModified': datetime.datetime(2019, 11, 8, 14, 34, 4, tzinfo=tzlocal()), u'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', u'StorageClass': 'STANDARD', u'Key': u'data/', u'Size': 0}


<code>Key</code> scheint den relativen Pfad zu einem Objekt darzustellen. Im folgenden wird eine Iteration über diesen Key vorgenommen.

In [14]:
i = 0
file_to_retrieve = 'clustering_result.csv'
for content_element in content_list:
    if file_to_retrieve in content_element['Key']: 
        print('{}: {}'.format(str(i), content_element['Key']))
    i += 1

1: data/clustering_result.csv


In [34]:
file_list = []
for content_element in content_list:
    if file_to_retrieve in content_element['Key']:
        file_list.append(content_element['Key'])
print('In Liste vorhandene Element: {}'.format(str(len(file_list))))

In Liste vorhandene Element: 1


In [44]:
s3_file = s3_client.get_object(Bucket=bucket_name, Key=file_list[0])
print('Type of object: {}'.format(str(type(s3_file))))

Type of object: <type 'dict'>


In [39]:
print(s3_file)

{u'Body': <botocore.response.StreamingBody object at 0x7f2de2aa8450>, u'AcceptRanges': 'bytes', u'ContentType': 'binary/octet-stream', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'i8fmF+XrzXb8N8bugs9nwaH+bE1fCb0jwcNg/G5lgfjdn+h8OoNkpiCRDb0MA/y7+0Tgz0eUvEc=', 'RequestId': '7E16E8E0A1130C40', 'HTTPHeaders': {'content-length': '19322227', 'x-amz-id-2': 'i8fmF+XrzXb8N8bugs9nwaH+bE1fCb0jwcNg/G5lgfjdn+h8OoNkpiCRDb0MA/y7+0Tgz0eUvEc=', 'accept-ranges': 'bytes', 'server': 'AmazonS3', 'last-modified': 'Sun, 26 Jan 2020 17:08:04 GMT', 'x-amz-request-id': '7E16E8E0A1130C40', 'etag': '"79ad4dde2e2c759c52243eecf613ed88-3"', 'date': 'Fri, 31 Jan 2020 15:29:25 GMT', 'content-type': 'binary/octet-stream'}}, u'LastModified': datetime.datetime(2020, 1, 26, 17, 8, 4, tzinfo=tzutc()), u'ContentLength': 19322227, u'ETag': '"79ad4dde2e2c759c52243eecf613ed88-3"', u'Metadata': {}}


In [45]:
result_file = s3_file['Body'].read()
print('Type of object: {}'.format(str(type(result_file))))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Normalisierung der importierten Daten
Da die Daten vorhanden sind können diese in einem <code>DataFrame</code> dargestellt werden.

In [32]:
df = pd.read_csv(io.BytesIO(result_file), header=0)

EmptyDataError: No columns to parse from file